<a href="https://colab.research.google.com/github/eclipseeyo/practiceML/blob/main/Encoder_decoder_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Практика по работе с Энкодерами

В этом задании мы потренируемся использовать различные энкодеры под наши задачи. Мы поработаем с классической бинарной классификацией отзывов на фильмы, попробуем усложнить и выйти на многоклассовую задачу, а после - подключим определение семантической близости фильмов по их описанию.

### Бинарная классификация

In [1]:
!pip install --upgrade datasets transformers

In [2]:
import numpy as np
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    set_seed,
)

SEED = 42
set_seed(SEED)

Перед вами практическое задание по бинарной классификации текстов с использованием предобученной модели BERT. Мы будем решать задачу определения тональности пользовательских отзывов на фильмы — **положительный** отзыв или **отрицательный**.

В качестве источника данных используем открытый датасет [IMDb](https://huggingface.co/datasets/stanfordnlp/imdb) от [Stanford NLP](https://nlp.stanford.edu/), содержащий 50 000 англоязычных рецензий, размеченных вручную. Цель — построить и обучить модель, способную автоматически классифицировать тексты по их эмоциональной окраске. Вы пройдёте все основные этапы пайплайна: от очистки текста и токенизации до обучения модели и оценки качества.

Перед тем, как мы двинемся дальше, проверьте включен или выключен GPU в colab. Работа с BERT потребует ресурсов, но предлагаем сначала заполнить весь необходимый код, а потом уже подключить GPU перед самым обучением модели. Советуем использовать доступную в colab Tesla T4.

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU на месте: {torch.cuda.get_device_name(0)}")

GPU на месте: Tesla T4


In [33]:
# датасет сразу подгружен в модуль datasets от HF

dataset = load_dataset("imdb")

In [34]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Мы видим, что в датасете уже подготовленное разделение данных на тренировочную и тестовую выборку, а также "unsupervised" набор - неразмеченная часть данных — то есть отзывы, у которых нет корректной метки (label не несёт обучающей информации).

* В части `train` и `test` у каждого отзыва есть метка: `label = 0` (негативный) или `label = 1` (позитивный).
* В части `unsupervised` колонка `label` есть, но её значение не используется — оно либо пустое, либо фиктивное (`-1`), так как эта часть предназначена для:

  * обучения без учителя (например, pretraining),
  * самостоятельного дообучения языковой модели,
  * полу- или слаборазмеченного обучения.

In [35]:
dataset["unsupervised"][0] # лейбл обозначен как "-1"

{'text': 'This is just a precious little diamond. The play, the script are excellent. I cant compare this movie with anything else, maybe except the movie "Leon" wonderfully played by Jean Reno and Natalie Portman. But... What can I say about this one? This is the best movie Anne Parillaud has ever played in (See please "Frankie Starlight", she\'s speaking English there) to see what I mean. The story of young punk girl Nikita, taken into the depraved world of the secret government forces has been exceptionally over used by Americans. Never mind the "Point of no return" and especially the "La femme Nikita" TV series. They cannot compare the original believe me! Trash these videos. Buy this one, do not rent it, BUY it. BTW beware of the subtitles of the LA company which "translate" the US release. What a disgrace! If you cant understand French, get a dubbed version. But you\'ll regret later :)',
 'label': -1}

В нашей задаче мы не будем использовать `unsupervised` часть — только размеченные данные из `train` и `test`. Точнее, только из `train` - 50.000 объектов на обучение это неплохо, но для учебной задачи долговато. С целью сокращения времени - сократим и датасет.

**Задание 1.** Давайте оставим только `train`, а затем перемешаем его с помщью `shuffle`, а после разобьем на обучающую и валидационную выборки в соотношении 80:20.

> Попробуем удержать баланс классов? Установите в гиперпараметре пропорциональное распределение классов. Если не помните, как это выполнить, загляните в описание метода [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

In [36]:
data = dataset["train"]
data = pd.DataFrame(data)
train_df, val_df = train_test_split(data, test_size=0.2, shuffle=True, stratify=data['label'], random_state=42)

In [37]:
train_df

,text,label
20022,I have always been a huge James Bond fanatic! ...,1
4993,I am a Christian and I say this movie had terr...,0
24760,"Neatly sandwiched between THE STRANGER, a smal...",1
13775,Years ago I did follow a soap on TV. So I was ...,1
20504,"Here's a gritty, get-the-bad guys revenge stor...",1
...,...,...
22580,Popular radio storyteller Gabriel No one(Robin...,1
20473,"Throughout this film, you might think this fil...",1
10468,Quite what the producers of this appalling ada...,0
5163,"""Trigger Man"" is definitely the most boring an...",0


**Вопрос 1.** Сколько отзывов положительного класса `(label = 1)` содержится в `val_df` после разбиения?

In [38]:
val_df['label'].value_counts()

,count
label,
1,2500
0,2500


**Задание 2.** **А стоит ли чистить отзывы?**

Перед тем как обучать модель на текстовых данных, важно понять, в каком виде приходят тексты, и нужна ли им предобработка.

В нашем случае отзывы взяты напрямую из `IMDb` — они написаны живыми пользователями, без фильтрации и нормализации. Чтобы принять решение об очистке, посмотрите на первые несколько примеров из датасета.

Обратите внимание на:

* наличие заглавных букв,
* пунктуацию (много ли её? нарушает ли она понимание?),
* цифры и HTML-теги (встречаются ли? нужны ли они для тональности?),
* стоп-слова (могут ли они мешать?).

In [39]:
train_df['text'][:10].to_numpy()

array(['I have always been a huge James Bond fanatic! I have seen almost all of the films except for Die Another Day, and The World Is Not Enough. The graphic\'s for Everything Or Nothing are breathtaking! The voice talents......... WOW! I LOVE PIERCE BROSNAN! He is finally Bond in a video game! HE IS BOND! I enjoyed the past Bond games: Goldeneye, The World Is Not Enough, Agent Under Fire, and Nightfire. This one is definitely the best! Finally, Mr. Brosnan, (may I call him Mr. Brosnan as a sign of respect? Yes I can!) He was phenomenally exciting to hear in a video game....... AT LONG LAST! DUH! I\'ve seen him perform with Robin Williams, and let me tell you, they make a great team. Pierce Brosnan is funny, wickedly handsome ( I mean to say wickedly in a good way,) and just one of those actor\'s who you would want to walk up to and wrap your arms around and hug, saying: "Pierce Brosnan, thank you for being James Bond," "If it wasn\'t for you, I wouldn\'t know who James Bond is." He\'

In [11]:
# посмотрите на первые десять примеров из датасета (можно взять и больше) и поизучайте их

# здесь ваш код
# (＠_＠)

**Вопрос 2** - на размышление: *Какую базовую предобработку вы бы применили к этим отзывам перед подачей в токенизатор BERT?*

> *Важно помнить*: модели `BERT` обучаются на сырых текстах, но легкая очистка всё же может помочь — особенно, если мы визуализируем текст, считаем TF-IDF или хотим сделать простой анализ.

В следующем шаге вы напишете функцию очистки текста. Если вы считаете, что некоторые из этих элементов обработки не нужны, всё равно, пожалуйста, выполните их. В бонусном задании вы сможете вернуться к этому вопросу и поэкспериментировать с разными подходами к обработке.

**Задание 3.** Теперь почистим наши тексты. Напишите функцию `clean_text`, которая будет очищать текст от лишнего. Вы можете опираться на код с семинара или написать свой вариант. Давайте: 1) приведем к нижнему регистру 2) уберем числа 3) удалим переносы строк 4) уберем стоп-слова.

Также удалите из таблицы те строки, где текст отзыва оказался пустым после очистки.

Вам здесь пригодятся модули `re`, `string`, `nltk` (`nltk.download('stopwords')`), `stopwords` из `nltk.corpus`. Не забудьте применить обработку и к трейну, и к тесту.

In [40]:
import re
import nltk
from nltk.corpus import stopwords

def clean_text(text):
  text = text.lower()
  text = re.sub('\n', ' ', text)
  text = re.sub('\d+', '', text)
  text = ' '.join([word for word in text.split() if word not in stop_words])
  return text

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

train_df['text'] = train_df['text'].apply(clean_text)
val_df['text'] = val_df['text'].apply(clean_text)

train_df.dropna(subset=['text'], inplace=True)
val_df.dropna(subset=['text'], inplace=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Вопрос 3.** Чему равна средняя длина очищенных отзывов (в символах) в `train_df`?

> Добавить блок с цитатой



In [41]:
train_df['text'].apply(len).mean()

np.float64(911.1124)

**Задание 4.** Теперь реализуем функцию оценки качества. При обучении модели через `Trainer` из библиотеки 🤗 можно передать свою функцию подсчёта метрик. Это позволяет отслеживать не только `loss`, но и, например, F1-score, accuracy или другие метрики качества.


Мы хотим реализовать функцию `compute_metrics`, которая будет передаваться в `Trainer`. Эта функция получает на вход кортеж `(logits, labels)` — предсказания модели и реальные метки, и должна возвращать **accuracy** (долю правильных ответов).

**Вопрос 4.** Почему, кстати, мы выбрали здесь именно **accuracy** метрику?



In [42]:
import numpy as np
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    pred = np.argmax(logits, axis=1)
    accuracy = accuracy_score(labels, pred)
    return {"accuracy" : accuracy}

Запустите следующий код. Если ваша функция написана верно, то код отработает без ошибок.

In [43]:
test_logits = np.array([[0.1, 0.9], [0.8, 0.2], [0.4, 0.6]])
test_labels = np.array([1, 0, 1])

eval_pred = (test_logits, test_labels)

metrics = compute_metrics(eval_pred)
print("Test metrics:", metrics)

assert "accuracy" in metrics, "Функция должна возвращать словарь с ключом 'accuracy'"
assert isinstance(metrics["accuracy"], float), "Значение accuracy должно быть числом"
assert abs(metrics["accuracy"] - 1.0) < 1e-6, "Ожидаемое значение accuracy: 1.0 на идеально предсказанном примере"

Test metrics: {'accuracy': 1.0}


**Задание 5.** Реализуем функцию токенизации текста. Перед тем как подавать тексты в модель `BERT`, нужно преобразовать их в числовой формат — токены. Это делает токенизатор модели.

Токенизатор превращает каждый отзыв в набор чисел, соответствующих подсловным единицам, которые `BERT` использует как вход. Но важно задать параметры, чтобы все примеры имели одинаковую длину и не обрезались неконтролируемо.


Реализуйте функцию `tokenize_function`, которая применяет токенизатор к колонке с очищенными отзывами. Мы будем использовать *построчную токенизацию с паддингом и усечением*.


In [44]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],  # Укажите название колонки с очищенным текстом
        padding="max_length",      # Дополняем последовательности до максимальной длины
        truncation=True,           # Обрезаем слишком длинные последовательности
        max_length=256,           # Максимальная длина (можно настроить)
    )

**Вопрос 5.** Какой набор параметров токенизатора будет наиболее подходящим для обучения модели классификации отзывов?

Теперь давайте немного порассуждаем насчет гиперпараметра `max_length`: какое количество токенов мы будет отправлять в нашу модель? Итак, что мы уже знаем:
1. максимум для `BERT` это `max_length = 512`.
2. В английском языке при использовании токенизации WordPiece (у `bert-base-uncased`, например) из 100 символов в среднем получается от 20 до 40 токенов, в зависимости от слов, пунктуации и наличия редких подслов.



In [45]:
# давайте проверим

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
text = train_df["text"].iloc[0]
tokens = tokenizer.tokenize(text)

print(len(text), "символов")
print(len(tokens), "токенов")

# 1127 символов
# 223 токенов

1553 символов
414 токенов


Если средняя длина отзывов после очистки — около 900 символов, то разумный выбор:

* `max_length = 256` — безопасный и быстрый вариант
* `max_length = 384` — если хочется сохранить больше контекста
* `max_length = 512` — максимум для **BERT**, но:

  * увеличивает время и потребление памяти
  * может быть избыточен (для коротких отзывов)



**Задание 6.** Подобрались мы и к обучению. Теперь, когда вы освоили все этапы предобработки, токенизации и настройки модели, давайте применим эти знания на практике и сравним три разных версии BERT на задаче классификации отзывов.



Это будет самая сложная часть - но и самая содержательная. Цель задания:

1. Написать **полный пайплайн обучения модели**, включая токенизацию (подтягиваем нашу функцию, написанную выше), обучение, валидацию.
2. Запустить пайнлайн с тремя предобученными моделями:
   * `"bert-base-uncased"`
   * `"roberta-base"`
   * `"distilbert-base-uncased"`
3. Обучить каждую модель на 3 эпохах и сравнить метрику качества (`accuracy`).

PS Вы можете написать логику для каждой модели отдельно, но будет более изящно, если вы объедините всё в единый цикл - без лишнего дублирования кода.

> Почитайте, что такое [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main_classes/data_collator#transformers.DataCollatorWithPadding:~:text=value%20at%20initialization.-,DataCollatorWithPadding,-class%20transformers.) и попробуйте интегрировать в своё решение.

In [46]:
# Аргументы обучения - пусть будут едины, чтобы мы согли синхронизировать результаты
args = TrainingArguments(
    output_dir="checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    warmup_ratio=0.1,
    weight_decay=0.01,
    fp16=True,
    report_to="none" # отключаем wandb и tensorboard - вы можете подключить, если это нужно
)

In [47]:
val_df.shape

(5000, 2)

In [48]:
from transformers import (
    AutoTokenizer,         # общий класс-фабрика для всех токенизаторов
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
)

# Задаём модели
model_names = {
    "BERT-base": "bert-base-uncased",
    "RoBERTa-base": "roberta-base",
    "DistilBERT-base": "distilbert-base-uncased",
}

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Обновленный цикл обучения
results = {}
for label, model_id in model_names.items():
    print(f"\n=== Начинаем обучение модели: {label} ({model_id}) ===")

    # Инициализация токенизатора и модели
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)

    # Токенизация данных - используем исправленную функцию
    tokenized_train = train_dataset.map(
        tokenize_function,  # Используем функцию напрямую
        batched=True,
        remove_columns=["text"]
    )

    tokenized_val = val_dataset.map(
        tokenize_function,  # Используем функцию напрямую
        batched=True,
        remove_columns=["text"]
    )

    # Обновляем токенизатор для коллатора
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # Инициализация тренера
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Обучение и оценка
    trainer.train()
    eval_res = trainer.evaluate()

    # Сохраняем результат
    results[label] = eval_res["eval_accuracy"]
    print(f"Модель {label} завершила обучение. Accuracy: {eval_res['eval_accuracy']:.4f}")


=== Начинаем обучение модели: BERT-base (bert-base-uncased) ===


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/tmp/ipython-input-48-3896526382.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/tmp/ipython-input-48-3896526382.py:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.324200,0.318925,0.893400
2,0.229100,0.363058,0.902000
3,0.082700,0.498530,0.902600


Модель DistilBERT-base завершила обучение. Accuracy: 0.9026


**Бонусное задание.** Выберете модель: можно взять из числа предложенных или любую другую - и попробуйте максимально увеличить метрики на валидации. Можно экспериментировать с методами обработки текстов, с архитектурами модели, подбором гиперпараметров и тд. Можно и учеличить объем данных - если вам позволит время и ресурсы.

Напишите о своих успехах нам на степике в комментариях под бонусным вопросом. И студентам, и нам будет очень интересно прочесть ваши эксперименты и результаты.

In [ ]:
# здесь ваш код
# ヽ(♡‿♡)ノ